### LCEL(대화내용 기억하기): 메모리 추가

임의 체인에 메모리 저장하기. 메모리 클래스를 사용하지만 수동으로 저장해야함.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("LCEL-ADD-CHAIN")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-ADD-CHAIN


In [25]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature= 0 
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)

대화 내용을 저장할 메모리인 ConversationBefferMemory 생성. return_messages 인자를 True로 설정하여, chain의 prompt에 입력될 수 있도록 key 값 설정

In [26]:
memory = ConversationBufferMemory(return_messages=True, memory_key = "chat_history")

In [27]:
print(memory.load_memory_variables({}))

{'chat_history': []}


chat_history 변수에 RunnableLambda를 통해 ConversationBefferMemory의 load_memory_variables 값을 할당. <Br>itemgetter 함수를 통해 chat_history 키에 해당하는 밸류 추출

In [28]:
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")
)

In [29]:
runnable.invoke({"input":"hi"})

{'input': 'hi', 'chat_history': []}

In [32]:
chain = runnable | prompt | model

In [34]:
response = chain.invoke({"input":"만나서 반갑습니다. 제 이름은 테디입니다."})
print(response.content)

안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?


메모리에는 히스토리가 저장되지 않은 모습. 수동으로 저장해줘야 하기 때문.

In [35]:
print(memory.load_memory_variables({}))

{'chat_history': []}


### 커스텀 ConversationChain 구현 예시

대화를 나누면 자동으로 memory에 저장하기 커스텀

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature= 0
    )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)

In [48]:
class CustomConversationChain(Runnable):
    def __init__(self, prompt, llm , memory, input_key = "input"):
        self.prompt = prompt
        self.llm = llm
        self.memory = memory
        self.input_key = input_key
        self.chain = (
            RunnablePassthrough.assign(
            chat_history = RunnableLambda(self.memory.load_memory_variables)
            | itemgetter(memory.memory_key)
            ) 
            | prompt 
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs = None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs = {"human": query}, outputs = {"ai": answer})
        return answer


In [49]:
conversation_chain = CustomConversationChain(prompt, model, memory)

In [50]:
conversation_chain.invoke("안녕하세요. 만나서 반갑습니다. 제 이름은 테디입니다.")

'안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [51]:
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='안녕하세요. 만나서 반갑습니다. 제 이름은 테디입니다.'), AIMessage(content='안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?')]}
